In [ ]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import random

In [ ]:
# Set up a blank English NLP object
nlp = spacy.blank('en')

# Add the NER pipeline to the pipeline of the NLP object
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

# Define the labels that the NER model will recognize
labels = ['PERSON', 'ORG', 'GPE']

# Add the labels to the NER model
for label in labels:
    ner.add_label(label)

In [ ]:
# Load your training data here
TRAIN_DATA = [...] # your training data

# Set up the optimizer and other training parameters
n_iter = 100
batch_size = 4
dropout = 0.5
optimizer = nlp.begin_training()

In [ ]:
# Start the training loop
for i in range(n_iter):
    losses = {}
    # Shuffle the training data to ensure that the model doesn't overfit to the order of the data
    random.shuffle(TRAIN_DATA)
    # Create batches of data using spaCy's minibatch function
    batches = minibatch(TRAIN_DATA, size=compounding(batch_size, 32, 1.001))
    for batch in batches:
        # Extract the text and annotations from the batch
        texts, annotations = zip(*batch)
        examples = []
        # Convert the text and annotations into Example objects
        for i in range(len(texts)):
            examples.append(Example.from_dict(nlp.make_doc(texts[i]), annotations[i]))
        # Update the NER model with the examples
        nlp.update(examples, sgd=optimizer, drop=dropout, losses=losses)

In [ ]:
# Print the loss every 10 iterations
    if i % 10 == 0:
        print(f"Loss at iteration {i}: {losses['ner']}")

# Save the trained NER model to disk
nlp.to_disk('/path/to/ner_model')